In [0]:
import torch
import pandas as pd

In [0]:
class OneHotEncoder():
  
  def __init__(self,series):
    unique_values = series.unique()
    self.ordinals = {c:i for i, c in enumerate(unique_values)}
    self.encoder = torch.eye(len(unique_values),len(unique_values))
    
  def __getitem__(self,value):
    return self.encoder[self.ordinals[value]]

In [3]:
look = pd.read_csv('./kc_house_data.csv')
look.iloc[0]

id                    7129300520
date             20141013T000000
price                     221900
bedrooms                       3
bathrooms                      1
sqft_living                 1180
sqft_lot                    5650
floors                         1
waterfront                     0
view                           0
condition                      3
grade                          7
sqft_above                  1180
sqft_basement                  0
yr_built                    1955
yr_renovated                   0
zipcode                    98178
lat                      47.5112
long                    -122.257
sqft_living15               1340
sqft_lot15                  5650
Name: 0, dtype: object

In [0]:
categorical = ['waterfront','view','condition','grade']

In [0]:
discard = ['id']

In [0]:
import dateutil

In [0]:
class DateEncoder():
  
  def __getitem__(self,datestring):
    parsed = dateutil.parser.parse(datestring)
    return torch.Tensor([parsed.year,parsed.month,parsed.day])

In [14]:
dates = ['date']
DateEncoder()['20141013T000000']

tensor([2014.,   10.,   13.])

In [0]:
from torch.utils.data import Dataset

In [0]:
class MixedCSV(Dataset):
  def __init__(self,datafile,output_series_name,date_series_names,categorical_series_names,ignore_series_names):
    self.dataset = pd.read_csv('./kc_house_data.csv')
    self.output_series_name = output_series_name
    self.encoders = {}
    for series_name in date_series_names:
      self.encoders[series_name] = DateEncoder()
    for series_name in categorical_series_names:
      self.encoders[series_name] = OneHotEncoder(self.dataset[series_name])
    self.ignore = ignore_series_names
   
  def __len__(self):
    return len(self.dataset)
   
  def __getitem__(self,idx):
    if type(idx) == torch.Tensor:
      idx = idx.item()
    sample = self.dataset.iloc[idx]
    output = torch.Tensor([sample[self.output_series_name]])
    
    input_components = []
    for name,value in sample.items():
      if name in self.ignore:
        continue
      elif name in self.encoders:
        input_components.append(self.encoders[name][value])
      else :
        input_components.append(torch.Tensor([value]))
     
    input = torch.cat(input_components)
    return input,output
  
      

In [25]:
houses = MixedCSV('./kc_house_data.csv','price',dates,categorical,discard)
houses[0]

(tensor([ 2.0140e+03,  1.0000e+01,  1.3000e+01,  2.2190e+05,  3.0000e+00,
          1.0000e+00,  1.1800e+03,  5.6500e+03,  1.0000e+00,  1.0000e+00,
          0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.1800e+03,  0.0000e+00,
          1.9550e+03,  0.0000e+00,  9.8178e+04,  4.7511e+01, -1.2226e+02,
          1.3400e+03,  5.6500e+03]), tensor([221900.]))